# 03 — Azure ML AutoML (Image Object Detection) Baseline
Submit an AutoML job for multi-label defect detection.

**Prerequisites**
- Azure ML Workspace + GPU compute
- SDK: azure-ai-ml
- MLTable + JSONL (train/val)


In [ ]:
# !pip install azure-ai-ml azure-identity
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient, Input
from azure.ai.ml.automl import image_object_detection
from azure.ai.ml.automl import ObjectDetectionPrimaryMetrics
import os

SUBSCRIPTION_ID = os.getenv('AZ_SUBSCRIPTION_ID','<sub-id>')
RG_NAME = os.getenv('AZ_RESOURCE_GROUP','<rg>')
WS_NAME = os.getenv('AZ_ML_WORKSPACE','<workspace>')

ml_client = MLClient(DefaultAzureCredential(), SUBSCRIPTION_ID, RG_NAME, WS_NAME)

train_input = Input(type='mltable', path='mltable/train')
val_input = Input(type='mltable/val')

job = image_object_detection(
    compute=os.getenv('AML_COMPUTE','gpu-cluster'),
    experiment_name='defects-automl',
    training_data=train_input,
    validation_data=val_input,
    target_column_name='label',
    primary_metric=ObjectDetectionPrimaryMetrics.MEAN_AVERAGE_PRECISION,
)
job.set_training_parameters(tile_grid_size='3x2', tile_overlap_ratio=0.25)
job.set_limits(timeout_minutes=180)
job.set_sweep(max_trials=12, max_concurrent_trials=4)

returned_job = ml_client.jobs.create_or_update(job)
print(f'Submitted: {returned_job.name}')
